# Standard Nystrom-based Spectral Clustering

This algorithm is from "[Fast Spectral Clustering via the Nystrom Method](http://www.cs.columbia.edu/~jebara/papers/ALT2013FSCVTNM.pdf)" by Choromanska et el.

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse.linalg
import scipy.cluster.vq

In [2]:
A = np.loadtxt('../data/processed/usps.csv', delimiter=',')

In [3]:
inds = A[:, -1] < 3
X = A[inds, :-2]
Y = A[inds, -1].astype(int)
n, d = X.shape
n, d

(2199, 255)

In [4]:
l = 100
inds = np.random.choice(d, l, replace=False)

In [5]:
W_hat = np.empty((n, l))
for i in range(n):
    for j in range(l):
        val = np.e ** (-1 * np.linalg.norm(X[i] - X[inds[j]]) ** 2)
        W_hat[i, j] = val

In [6]:
D = np.diag(1 / np.sqrt(W_hat.sum(axis=1)))
Delta = np.diag(1 / np.sqrt(W_hat.sum(axis=0)))

In [7]:
C = np.identity(n)[:, inds] - np.sqrt(l / n) * D.dot(W_hat).dot(Delta)

In [8]:
B = C[inds, :]

In [9]:
r = l // 2
U, s, V = sp.sparse.linalg.svds(B, r)
B_r = U.dot(np.diag(s)).dot(V)

In [10]:
s_Br, U_Br = np.linalg.eigh(B_r)
S_Br = np.diag(s_Br)

In [11]:
S_tilde = (n / l) * S_Br
U_tilde = np.sqrt(n / l) * C.dot(U_Br).dot(np.linalg.pinv(S_Br))

In [12]:
k = 2
Z = U_tilde[:, :k]
z = np.sqrt((Z ** 2).sum(axis=1))
for i in range(n):
    if z[i] > 0:
        Z[i] /= z[i]

In [13]:
centroids, distortion = sp.cluster.vq.kmeans(Z, k)
centroids

array([[ 0.00325986,  0.98691578],
       [-0.26022793, -0.74821877]])

In [14]:
y_hat = np.zeros(n, dtype=int)
for i in range(n):
    dists = np.array([np.linalg.norm(Z[i] - c) for c in centroids])
    y_hat[i] = np.argmin(dists) + 1

In [15]:
accuracy = np.zeros(k)
for i in range(k):
    y_hat = (y_hat + i) % (k + 1)
    y_hat[y_hat == 0] = 1
    accuracy[i] = (Y == y_hat).sum() / n * 100
accuracy.max()

83.492496589358794